# Airflow
- 유연한 파이썬 프레임워크를 사용해 쉽게 데이터 파이프라인 구축 가능 
- 파이프라인이나 워크플로 태스크를 DAG로 정의 
- DAG 파일 안에 파이썬 코드를 사용하여 DAG 정의
- 외부 데이터베이스, 빅데이터 기술 등 여러 시스템 간에 데이터 프로세스 결합할 숭 ㅣㅆ는 복잡한 데이터 파이프라인 구축 가능

## 데이터 파이프라인
- 원하는 결과를 얻기 위해 실행되는 여러 테스크 또는 동작으로 구성

- 서로 다른 테스크로 구성되고 각 테스크는 정해진 순서대로 진행되어야 함

- 방향성 비순환 그래프(DAG, Directed Acyclic Graph) : 명확한 방향성 존재. 태스크 그래프 효율적 해결 및 실행하기 위해 사용

- DAG 구성 : 다음 태스크로 향하기 전 이전 태스크가 완료되었는 지 확인 => 실행 대기열에 있는 태스크 실행 => 태스크 수행이 완료되면 완료 표시 => 그래프의 모든 태스크가 완료될 때까지 1단계로 돌아감

## 파이프라인 스케줄링 및 실행

- Airflow 스케줄러 : DAG 분석 및 현재 시점에서 DAG 스케줄 지난 경우 Airflow 워커에 DAG 태스크 예약
- Airflow 워커 : 예약된 태스크 선택 및 실행
- Airflow 웹 서버 : 스케줄러에서 분석한 DAG 시각화 및 실행과 결과 확인 인터페이스 제공

# 예시
- 로켓 발사

## 데이터 탐색

1. curl
- cURL은 command line 기반의 웹 요청 도구이다. 
- Unix, Linux, Windows 등의 주요 OS에서 구동 가능하며 HTTP/HTTPS/FTP/LDAP/SCP/TELNET/SMTP/POP3 등 핵심 프로코콜을 지원하기 때문에 유용하게 사용된다.
- download와 upload가 모두 가능하다.
- 출처 : https://www.leafcats.com/188

2. Airflow
- Airflow의 장점은 하나 이상의 단계로 구성된 대규모 작업을 개별 태스크로 분할하고 DAG로 형성할 수 있는 것

In [5]:
# Launch Library API에 대한 curl 요청 및 응답
!curl -L "https://ll.thespacedevs.com/2.2.0/launch/upcoming"

{"count":249,"next":"https://ll.thespacedevs.com/2.2.0/launch/upcoming/?limit=10&offset=10","previous":null,"results":[{"id":"a6b9deb4-f78d-4b57-8e47-98c5aea99d9e","url":"https://ll.thespacedevs.com/2.2.0/launch/a6b9deb4-f78d-4b57-8e47-98c5aea99d9e/","slug":"falcon-9-block-5-starlink-group-4-26","name":"Falcon 9 Block 5 | Starlink Group 4-26","status":{"id":3,"name":"Launch Successful","abbrev":"Success","description":"The launch vehicle successfully inserted its payload(s) into the target orbit(s)."},"last_updated":"2022-08-10T02:32:29Z","net":"2022-08-10T02:14:00Z","window_end":"2022-08-10T02:14:00Z","window_start":"2022-08-10T02:14:00Z","probability":70,"holdreason":"","failreason":"","hashtag":null,"launch_service_provider":{"id":121,"url":"https://ll.thespacedevs.com/2.2.0/agencies/121/","name":"SpaceX","type":"Commercial"},"rocket":{"id":7580,"configuration":{"id":164,"url":"https://ll.thespacedevs.com/2.2.0/config/launcher/164/","name":"Falcon 9","family":"Falcon","full_name":"F

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0

100 22311  100 22311    0     0  20468      0  0:00:01  0:00:01 --:--:-- 20468


## Airflow DAG 작성

1. pathlib
- pathlib 모듈의 기본 아이디어는 파일시스템 경로를 단순한 문자열이 아니라 객체로 다루자는 것
- 연산자를 새롭게 정의할 수 있게 되었다는 점
- 출처 : https://brownbears.tistory.com/415

In [6]:
import json
import pathlib

import airflow.utils.dates
import requests
import requests.exceptions as requests_exceptions
from airflow import DAG
from airflow.operators.bash import BashOperator
from airflow.operators.python import PythonOperator

# DAG 객체 인스턴스 생성 
dag = DAG( # DAG 클래스는 두 개의 인수가 필요 
    dag_id="download_rocket_launches", # Airflow UI에 표시되는 DAG 이름 
    description="Download rocket pictures of recently launched rockets.",
    start_date=airflow.utils.dates.days_ago(14), # DAG 처음 실행 시작 날짜
    schedule_interval=None, # DAG 실행 간격, None으로 설정해 DAG가 자동으로 실행되지 않음 
)

download_launches = BashOperator( # BashOperator 이용해 curl로 URL 결과값 다운로드
    task_id="download_launches", # 태스크 이름 
    bash_command="curl -o /tmp/launches.json -L 'https://ll.thespacedevs.com/2.0.0/launch/upcoming'",  # noqa: E501
    dag=dag, # DAG 변수에 대한 참조 
)

# 결과값 파싱 및 모든 로켓 사진 다운로드 
def _get_pictures(): # 호풀 파이썬 함수 
    # Ensure directory exists
    pathlib.Path("/tmp/images").mkdir(parents=True, exist_ok=True) # 경로가 없으면 디렉토리 생성

    # Download all pictures in launches.json
    with open("/tmp/launches.json") as f: # 로켓 발사 JSON 파일 열기 
        launches = json.load(f) # 딕셔너리로 읽기
        image_urls = [launch["image"] for launch in launches["results"]] # 모든 발사에 대한 'image'의 URL 값 읽기
        for image_url in image_urls: # 모든 이미지 URL을 얻기 위한 루프 
            try:
                response = requests.get(image_url) # 이미지 가져오기 
                image_filename = image_url.split("/")[-1] # 마지막 파일 이름만 가져오기
                target_file = f"/tmp/images/{image_filename}" # 타겟 파일 저장 경로 구성
                with open(target_file, "wb") as f: # 타겟 파일 핸들 열기
                    f.write(response.content) # 파일 경로에 이미지 쓰기 
                print(f"Downloaded {image_url} to {target_file}") # 결과 출력 
            except requests_exceptions.MissingSchema:
                print(f"{image_url} appears to be an invalid URL.")
            except requests_exceptions.ConnectionError:
                print(f"Could not connect to {image_url}.")


get_pictures = PythonOperator(
    task_id="get_pictures", 
    python_callable=_get_pictures, #  인수에 호출이 가능한 일반 함수 가리킴 
    dag=dag
)

notify = BashOperator(
    task_id="notify",
    bash_command='echo "There are now $(ls /tmp/images/ | wc -l) images."',
    dag=dag,
)

download_launches >> get_pictures >> notify

ModuleNotFoundError: No module named 'airflow'

# Airflow DAG 실행

1. shutil
- Windows에서 파일 복사하기 위해 사용
- 출처 : https://code.tutsplus.com/ko/tutorials/file-and-directory-operations-using-python--cms-25817

2. os 
- dag 폴더 생성 위해 사용

In [12]:
import shutil
import os
# !pip install apache-airflow
# !airflow db init
# !airflow user create --username admin --password admin --firstname Anonymous --lastname Adminn --role Admin --eamil addmin@example.org
# os.mkdir('C:/Users/canmanmo/airflow/dags')
# shutil.copy('C:/Users/canmanmo/Desktop/data-pipelines-with-apache-airflow-main/chapter02/dags/download_rocket_launches.py', 'C:/Users/canmanmo/airflow/dags/')
!airflow webserver
!airflow scheduler

Traceback (most recent call last):
  File "C:\Users\canmanmo\anaconda3\envs\airflow_book\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\canmanmo\anaconda3\envs\airflow_book\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\canmanmo\anaconda3\envs\airflow_book\Scripts\airflow.exe\__main__.py", line 7, in <module>
  File "C:\Users\canmanmo\anaconda3\envs\airflow_book\lib\site-packages\airflow\__main__.py", line 38, in main
    args.func(args)
  File "C:\Users\canmanmo\anaconda3\envs\airflow_book\lib\site-packages\airflow\cli\cli_parser.py", line 50, in command
    func = import_string(import_path)
  File "C:\Users\canmanmo\anaconda3\envs\airflow_book\lib\site-packages\airflow\utils\module_loading.py", line 32, in import_string
    module = import_module(module_path)
  File "C:\Users\canmanmo\anaconda3\envs\airflow_book\lib\importlib\__init__.py", line 127, in import_module
    return _bootstrap._gcd_import(name[l

# Windows 10에서 pwd 라이브러리 실패로 다른 환경 구축!!

# Reference

1. 블로그(검색어)
- https://www.leafcats.com/188 (curl 명령어)
- https://brownbears.tistory.com/415 (pathlib)
- https://code.tutsplus.com/ko/tutorials/file-and-directory-operations-using-python--cms-25817 (python window dir copy)